# How to present CV with Neptune

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/how-to-guides/present-cross-validation-results/notebooks/Neptune_cross_validation.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>
<a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/how-to-guides/present-cross-validation-results/notebooks/Neptune_cross_validation.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>
<a target="_blank" href="https://app.neptune.ai/o/common/org/showroom/runs/details?viewId=standard-view&detailsTab=metadata&shortId=SHOW-27624"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>
<a target="_blank" href="https://docs.neptune.ai/tutorials/tracking_cross_validation_results/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

When training models with cross-validation, you can use Neptune namespaces to organize, visualize and compare models.

By the end of this guide, you will learn how to organize your run to track cross-validation metadata, so that you can easily analyze the results.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

This example assumes:
  - Basic familiarity with Neptune ([create run and log metadata to it](https://docs.neptune.ai/you-should-know/what-can-you-log-and-display)),
  - Familiarity with cross-validation techniques in machine learning.




## Install Neptune and dependencies

In [ ]:
! pip install -U neptune scikit-learn torch torchvision tqdm

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags=["cross-validation"],
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. 

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

Connect your script to Neptune application and create new run.

In [ ]:
import neptune

run = neptune.init_run(
    project="common/showroom",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags=["cross-validation"],
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

## Log config and hyperparameters

### Log Hyperparameters

In [ ]:
parameters = {
    "epochs": 2,
    "learning_rate": 1e-2,
    "batch_size": 10,
    "image_size": (3, 32, 32),
    "n_classes": 10,
    "k_folds": 2,
    "checkpoint_name": "checkpoint.pth",
    "dataset_size": 1000,
    "seed": 42,
}

In [ ]:
from neptune.utils import stringify_unsupported

run["parameters"] = stringify_unsupported(parameters)

### Log Config
Model and Dataset

In [ ]:
from functools import reduce

image_size = reduce(lambda x, y: x * y, parameters["image_size"])

In [ ]:
import torch.nn as nn


class BaseModel(nn.Module):
    def __init__(self, input_sz, hidden_dim, n_classes):
        super(BaseModel, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_sz, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, n_classes),
        )

    def forward(self, input):
        x = input.view(-1, image_size)
        return self.main(x)

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
run["parameters/device"] = str(device)

In [ ]:
torch.manual_seed(parameters["seed"])

model = BaseModel(
    image_size,
    image_size,
    parameters["n_classes"],
).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=parameters["learning_rate"])

Log model, criterion and optimizer name

In [ ]:
run["parameters/model/name"] = type(model).__name__
run["parameters/model/criterion"] = type(criterion).__name__
run["parameters/model/optimizer"] = type(optimizer).__name__

In [ ]:
from torchvision import datasets, transforms

data_tfms = {
    "train": transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
}

In [ ]:
dataset = datasets.FakeData(
    size=parameters["dataset_size"],
    image_size=parameters["image_size"],
    num_classes=parameters["n_classes"],
    transform=data_tfms["train"],
)

Log dataset details

In [ ]:
run["dataset/transforms"] = stringify_unsupported(data_tfms)
run["dataset/size"] = parameters["dataset_size"]

## Log losses and metrics per fold 
Training Loop

In [ ]:
from sklearn.model_selection import KFold

splits = KFold(n_splits=parameters["k_folds"], shuffle=True)
epoch_acc_list, epoch_loss_list = [], []

In [ ]:
from torch.utils.data import SubsetRandomSampler, DataLoader
from tqdm.auto import tqdm, trange

for fold, (train_ids, _) in tqdm(enumerate(splits.split(dataset))):
    train_sampler = SubsetRandomSampler(train_ids)
    train_loader = DataLoader(dataset, batch_size=parameters["batch_size"], sampler=train_sampler)
    for _ in trange(parameters["epochs"]):
        epoch_acc, epoch_loss = 0, 0.0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model.forward(x)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, y)
            acc = (torch.sum(preds == y.data)) / len(x)

            # Log batch loss and acc
            run[f"fold_{fold}/training/batch/loss"].append(loss)
            run[f"fold_{fold}/training/batch/acc"].append(acc)

            loss.backward()
            optimizer.step()

        epoch_acc += torch.sum(preds == y.data).item()
        epoch_loss += loss.item() * x.size(0)

    epoch_acc_list.append((epoch_acc / len(train_loader.sampler)) * 100)
    epoch_loss_list.append(epoch_loss / len(train_loader.sampler))

    # Log model checkpoint
    torch.save(model.state_dict(), f"./{parameters['checkpoint_name']}")
    run[f"fold_{fold}/checkpoint"].upload(parameters["checkpoint_name"])
    run.sync()

In [ ]:
from statistics import mean

# log mean acc and loss
run["results/metrics/train/mean_acc"] = mean(epoch_acc_list)
run["results/metrics/train/mean_loss"] = mean(epoch_loss_list)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Explore the run in Neptune

After starting a run, you will get a link on the cell output similar to https://app.neptune.ai/o/common/org/showroom/e/SHOW-27624 with: 
* **common/showroom** replaced by **your_workspace/your_project**,
* **SHOW-27624** replaced by your *run ID*. 

**Click on the link to open the run in Neptune app.**

<img src="https://neptune.ai/wp-content/uploads/metadata-in-neptune-5.gif" alt="Drawing" style="width: 100%;"/>
<center><small>Analysing per-fold metadata</small></center>

## Conclusion

You learned how to organize your run to track cross-validation metadata with Neptune and how to present the result in the Neptune app for further comparison and analysis. 

Visit our docs for more tutorials and guides on how to use Neptune: https://docs.neptune.ai
